# Global ensitivity Analysis
## Project initialisation

In [1]:
from brightway2 import *
from __future__ import unicode_literals, print_function

Using environment variable BRIGHTWAY2_DIR for data directory:
/home/jovyan/data


In [2]:
projects.set_current("Global Sensitivity Analysis demo")

Adding some uncertainties in sample data (from [this notebook](http://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/Getting%20Started%20with%20Brightway2.ipynb))

In [3]:
bw2setup()

Biosphere database already present!!! No setup is needed


In [4]:
import zipfile
import os
from bw2data.utils import download_file

filepath = download_file("forwast.bw2package.zip", url="http://lca-net.com/wp-content/uploads/")
dirpath = os.path.dirname(filepath)
zipfile.ZipFile(filepath).extractall(dirpath)
BW2Package.import_file(os.path.join(dirpath, "forwast.bw2package"))

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:04


Title: Writing activities to SQLite3 database:
  Started: 03/23/2017 11:28:04
  Finished: 03/23/2017 11:28:08
  Total time elapsed: 00:00:04
  CPU %: 68.30
  Memory %: 1.82


[Brightway2 SQLiteBackend: forwast]

## Adding uncertainties

In [5]:
from bw2data.utils import uncertainify
from stats_arrays import NormalUncertainty
uncertain_db = Database("forwast uncertain +")
uncertain_db.write(
    uncertain_db.relabel_data(
        uncertainify(
            Database("forwast").load(), 
            NormalUncertainty
        ), 
        "forwast uncertain +" 
    )
)

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:04


Title: Writing activities to SQLite3 database:
  Started: 03/23/2017 11:28:10
  Finished: 03/23/2017 11:28:15
  Total time elapsed: 00:00:04
  CPU %: 70.00
  Memory %: 1.70


In [120]:
method_key=('ReCiPe Endpoint (I,A)', 'ecosystem quality', 'total')#'IMPACT 2002+ (Endpoint)', 'climate change', 'total')
activity = uncertain_db.search('Wood products EU27')[0]
lca = LCA({activity: 1}, method_key)

## Random initialization
Initialization of the RNG stuff in the same way that in the MonteCarloLCA class

In [121]:
lca.load_lci_data()
from stats_arrays.random import MCRandomNumberGenerator
tech_rng = MCRandomNumberGenerator(lca.tech_params)
bio_rng = MCRandomNumberGenerator(lca.bio_params)
lca.load_lcia_data()
cf_rng = MCRandomNumberGenerator(lca.cf_params)

## Defining the problem data
Here, we want to define the parameters to sample using a list of selected elements of the biosphere and technosphere matrices

In [122]:
from lsa import lsa
rsca_summary, rscb_summary = lsa(lca)

In [123]:
import numpy as np
A_indices = np.array(np.array(rsca_summary)[1:3,1:3],int).tolist()
B_indices = np.array(np.array(rscb_summary)[1:3,1:3],int).tolist()
rev_activity, rev_product, rev_bio = lca.reverse_dict()
morris_problem = {
    'num_vars':len(A_indices)+len(B_indices),
    'names':[],
    'bounds':[],
    'groups':None
}
morris_problem['names'] += [
    'Technosphere '+ str(Database(rev_activity[x[0]][0]).get(rev_activity[x[0]][1])) + ' x ' + str(Database(rev_product[x[1]][0]).get(rev_product[x[1]][1]))
    for x in A_indices
    ]
morris_problem['names'] += [
    'Biosphere '+str(Database(rev_bio[x[0]][0]).get(rev_bio[x[0]][1])) + ' x ' + str(Database(rev_activity[x[1]][0]).get(rev_activity[x[1]][1]))
    for x in B_indices
    ]
# retrieve min and max values for populating morris_problem['bounds']
from pandas import DataFrame
tech_df = DataFrame.from_records(lca.tech_params)
bio_df = DataFrame.from_records(lca.bio_params)
morris_problem['bounds'] = [tech_df[(tech_df.type==1) & (tech_df.row==a_ij[0]) & (tech_df.col==a_ij[1])][['minimum','maximum']].values[0] for a_ij in A_indices] + [bio_df[(bio_df.type==2) & (bio_df.row==b_ij[0]) & (bio_df.col==b_ij[1])][['minimum','maximum']].values[0] for b_ij in B_indices]

## Computing Sobol indices
This class will generate the Morris sample and perform lcia computations using multiprocessing feature.

In [173]:
from SALib.sample import morris as ms
from SALib.analyze import morris as ma
import multiprocessing
import pyprind
import math

class ParallelGSALCA:
    def __init__(self, demand, method, morris_problem):
        self.demand = demand
        self.method = method
        # TODO generate morris_problem instead of parameter
        self.morris_problem = morris_problem
    def single_worker(self, sample):
        lca = LCA(self.demand, self.method)
        lca.load_lci_data()
        for a_ij, value in zip(self.A_indices, sample[:len(self.A_indices)]):
            lca.technosphere_matrix[tuple(a_ij)] = value
        for b_ij, value in zip(self.B_indices, sample[len(self.A_indices):]):
            lca.biosphere_matrix[tuple(b_ij)] = value
        lca.lci()
        lca.lcia()
        return lca.score
    def gsa(self, A_indices, B_indices, number_of_trajectories, cpus=None, chunk_size=None):
        self.A_indices = A_indices
        self.B_indices = B_indices
        cpus = cpus or multiprocessing.cpu_count()
        # TODO propose an automatic number_of_trajectories
        samples = ms.sample(self.morris_problem, number_of_trajectories, num_levels=4, grid_jump=2)
        if chunk_size:
            self.chunk_size = chunk_size
        else:
            self.chunk_size = max(cpus, number_of_trajectories // 100)
        pool = multiprocessing.Pool(processes=cpus)
        # TODO progress bar doesn't display as excepted
        #bar = pyprind.ProgBar(max(1,math.ceil(number_of_trajectories/self.chunk_size)))
        chunks = []
        scores = []
        for sample in samples:
            #chunks.append((self.demand, self.method, A_indices, B_indices, sample))
            chunks.append(sample)
            if len(chunks) == self.chunk_size:
                scores+=pool.map(self.single_worker, chunks)
                #bar.update()
                chunks=[]
        return scores

In [174]:
gsalca = ParallelGSALCA({activity: 1}, method_key, morris_problem)
scores = gsalca.gsa(A_indices, B_indices, number_of_trajectories = 100)

In [175]:
ma.analyze(morris_problem,samples,np.array(scores))

{'mu': array([ 0.,  0.,  0.,  0.]),
 'mu_star': array([ 0.,  0.,  0.,  0.]),
 'mu_star_conf': [0.0, 0.0, 0.0, 0.0],
 'names': ["Technosphere '_32 Wood products, except furniture, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003']) x '_32 Wood products, except furniture, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])",
  "Technosphere '_32 Wood products, except furniture, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003']) x '_32 Wood products, except furniture, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])",
  "Biosphere 'Sulfur dioxide' (kilogram, None, ('air',)) x '_75 Electricity, steam and hot water, EU27' (kilowatt hour, GLO, ['Input Output', 'EU27 2003'])",
  "Biosphere 'Carbon dioxide, fossil' (kilogram, None, ('air',)) x '_75 Electricity, steam and hot water, EU27' (kilowatt hour, GLO, ['Input Output', 'EU27 2003'])"],
 'sigma': array([ 0.,  0.,  0.,  0.])}